# IMPORTING LIBRARIES

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specific

# READING DATA

In [2]:
#reading Train Data 
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
#get data information 
print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# FILLING MISSING VALUES

In [3]:
#Droping columns wich have more than 80% missing value 

#Missing values in PoolQC
print('Missing values in PoolQC:',int((1-sum(data['PoolQC'].value_counts())/data.shape[0])*100),'%')

#Missing values in Alley
print('Missing values in Alley:',int((1-sum(data['Alley'].value_counts())/data.shape[0])*100),'%')

#Missing values in Fence
print('Missing values in Fence:',int((1-sum(data['Fence'].value_counts())/data.shape[0])*100),'%')

#Missing values in MiscFeature
print('Missing values in MiscFeature:',int((1-sum(data['MiscFeature'].value_counts())/data.shape[0])*100),'%')

#Dropping Columns
data = data.drop(['PoolQC', 'Alley','MiscFeature','Fence'], axis=1)
data_test = data_test.drop(['PoolQC', 'Alley','MiscFeature','Fence'], axis=1)

Missing values in PoolQC: 99 %
Missing values in Alley: 93 %
Missing values in Fence: 80 %
Missing values in MiscFeature: 96 %


In [4]:
#Detect the columns with missing values
dict_ = data.isna().sum().to_dict()
for key in dict_:
    if dict_[key] != 0:
        print(key,'contains',dict_[key],' missing value','\t(',dict_[key]/data.shape[0]*100,'%)')
        

LotFrontage contains 259  missing value 	( 17.73972602739726 %)
MasVnrType contains 8  missing value 	( 0.547945205479452 %)
MasVnrArea contains 8  missing value 	( 0.547945205479452 %)
BsmtQual contains 37  missing value 	( 2.5342465753424657 %)
BsmtCond contains 37  missing value 	( 2.5342465753424657 %)
BsmtExposure contains 38  missing value 	( 2.6027397260273974 %)
BsmtFinType1 contains 37  missing value 	( 2.5342465753424657 %)
BsmtFinType2 contains 38  missing value 	( 2.6027397260273974 %)
Electrical contains 1  missing value 	( 0.0684931506849315 %)
FireplaceQu contains 690  missing value 	( 47.26027397260274 %)
GarageType contains 81  missing value 	( 5.5479452054794525 %)
GarageYrBlt contains 81  missing value 	( 5.5479452054794525 %)
GarageFinish contains 81  missing value 	( 5.5479452054794525 %)
GarageQual contains 81  missing value 	( 5.5479452054794525 %)
GarageCond contains 81  missing value 	( 5.5479452054794525 %)


In [5]:
#Get the categorial features
num_features = [col for col in data.columns if data[col].dtypes !='O']
cat_features = [col for col in data.columns if col not in num_features]#categorical features in train data
noncat_features = [col for col in data.columns if col  in num_features] #categorical features in train data

num_features_test = [col for col in data_test.columns if data_test[col].dtypes !='O']
cat_features_test = [col for col in data_test.columns if col not in num_features_test] #categorical features in test data
noncat_features_test = [col for col in data_test.columns if col  in num_features_test] #categorical features in test data

#Filling the non-categorical missing values with the mean

#in train data
for feat in noncat_features:
    mean_value=data[feat].mean()
    data[feat].fillna(value=mean_value, inplace=True)

#in test data
for feat in noncat_features_test:
    mean_value=data_test[feat].mean()
    data_test[feat].fillna(value=mean_value, inplace=True)

#Filling the categorical missing values with the Mode
#cols = ["MSZoning","MasVnrType","MasVnrArea", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1" ,"BsmtFinType2", "Electrical", "GarageType", "GarageYrBlt","FireplaceQu", "GarageFinish" ,"GarageQual", "GarageCond" ]
data[cat_features]=data[cat_features].fillna(data.mode().iloc[0])
data_test[cat_features_test]=data_test[cat_features_test].fillna(data_test.mode().iloc[0])


In [6]:
#checking
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [7]:
def encodeCategories(Data,column):
    Data[column] = pd.Categorical(Data[column])
    Data[column] = Data[column].cat.codes
    return Data

max_ = data['SalePrice'].max()
min_ = data['SalePrice'].min()
data[noncat_features] = data[noncat_features].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
data_test[noncat_features_test] = data_test[noncat_features_test].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#encode the categorical data
#in train data
for feature in cat_features:
    data = encodeCategories(data,feature)
    
data = encodeCategories(data,'MSSubClass')
data = encodeCategories(data,'OverallQual')
data = encodeCategories(data,'OverallCond')
    
data.MSSubClass = pd.Categorical(data.MSSubClass)
data.OverallQual = pd.Categorical(data.OverallQual)
data.OverallCond = pd.Categorical(data.OverallCond)

data.GarageYrBlt = pd.Categorical(data.GarageYrBlt)
data.YrSold = pd.Categorical(data.YrSold)
data.YearRemodAdd = pd.Categorical(data.YearRemodAdd)
data.YearBuilt = pd.Categorical(data.YearBuilt)

#in test data
for feature in cat_features_test:
    data_test = encodeCategories(data_test,feature)  
    
data_test = encodeCategories(data_test,'MSSubClass')
data_test = encodeCategories(data_test,'OverallQual')
data_test = encodeCategories(data_test,'OverallCond')

data_test.MSSubClass = pd.Categorical(data_test.MSSubClass)
data_test.OverallQual = pd.Categorical(data_test.OverallQual)
data_test.OverallCond = pd.Categorical(data_test.OverallCond)

data_test.GarageYrBlt = pd.Categorical(data_test.GarageYrBlt)
data_test.YrSold = pd.Categorical(data_test.YrSold)
data_test.YearRemodAdd = pd.Categorical(data_test.YearRemodAdd)
data_test.YearBuilt = pd.Categorical(data_test.YearBuilt)

# Data Split

In [8]:
X = data.iloc[:,:data.shape[1]-1]
y = data.iloc[:,data.shape[1]-1:]
#splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Training

In [9]:
regr = SVR(C=1.0, epsilon=0.2)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# Mean squared error

In [10]:
print('MSE:',mean_squared_error(y_test, y_pred))

MSE: 0.018316039245957896


# Test data price prediction 

In [11]:
price = regr.predict(data_test)
price = (price*(max_ - min_))+ min_
data_test['predicted Price'] = price
data_test.to_csv('predicted price.csv')
